In [2]:
# unzip input file first

import xml.etree.ElementTree as ET
import requests
import math
import pprint
import time
import numpy as np
import pandas as pd
import os
import collections

In [55]:
# Read all census tracts and put points in dict
CENSUS_TRACTS_DIR = "census_tracts"
census_tracts_points = {}
for fname in os.listdir(CENSUS_TRACTS_DIR):
    latitudes = []
    longitudes = []
    try:
        # Parse the XML file
        root = ET.parse("{}/{}".format(CENSUS_TRACTS_DIR, fname)).getroot()
        for type_tag in root.findall("node"):
            d = type_tag.attrib
            latitudes.append(float(d["lat"]))
            longitudes.append(float(d["lon"]))
        census_tracts_points[fname.strip(".txt")] = list(zip(latitudes, longitudes))
        census_tracts_bounding[fname.strip(".txt")] = [(min(latitudes), min(longitudes)), (max(latitudes), max(longitudes))]
    except:
        print(fname)

18_04.txt
17_69.txt
17_41.txt
17_55.txt
22_11.txt
17_82.txt
17_83.txt
22_10.txt
17_54.txt
17_40.txt
17_68.txt
18_11.txt
18_05.txt
18_39.txt
3_02.txt
20_03.txt
16_02.txt
18_13.txt
17_56.txt
17_42.txt
22_12.txt
17_81.txt
17_80.txt
22_07.txt
17_57.txt
9800.txt
18_06.txt
18_12.txt
16_03.txt
20_02.txt
3_05.txt
14_02.txt
17_53.txt
17_47.txt
17_84.txt
22_02.txt
17_85.txt
1_01.txt
17_46.txt
17_52.txt
14_03.txt
12.txt
16_06.txt
18_17.txt
3_04.txt
3_06.txt
20_05.txt
18_29.txt
16_04.txt
10.txt
13_08.txt
14_01.txt
17_50.txt
17_78.txt
22_01.txt
17_86.txt
17_79.txt
17_51.txt
17_45.txt
1_02.txt
11.txt
16_05.txt
18_28.txt
20_04.txt
3_07.txt
9_01.txt
24_07.txt
24_13.txt
17_22.txt
17_37.txt
24_12.txt
9_02.txt
18_58.txt
18_64.txt
24_10.txt
23_19.txt
21_08.txt
19_18.txt
19_19.txt
21_09.txt
24_11.txt
23_18.txt
18_59.txt
18_61.txt
18_49.txt
24_29.txt
23_08.txt
17_18.txt
17_19.txt
19_08.txt
24_28.txt
18_48.txt
18_60.txt
18_62.txt
24_02.txt
17_33.txt
24_03.txt
18_63.txt
18_46.txt
24_26.txt
24_32.txt
23_13.txt

## Travis County Boundary API

In [3]:
# root = ET.parse('input-files/RoughlyTravisCounty.osm').getroot()

In [57]:
latitudes = []
longitudes = []
# for type_tag in root.findall("node"):
#     d = type_tag.attrib
#     latitudes.append(float(d["lat"]))
#     longitudes.append(float(d["lon"]))
# min_lat_travis, max_lat_travis, min_lon_travis, max_lon_travis = float("inf"), float("-inf"), float("inf"), float("-inf")
for tract_name in census_tracts_points:
    for lat, lon in census_tracts_points[tract_name]:
#         if lat < min_lat_travis:
#             min_lat_travis = lat
#         if lon < min_lon_travis:
#             min_lon_travis = lon
#         if lat > max_lat_travis:
#             max_lat_travis = lat
#         if lon > max_lon_travis:
#             max_lon_travis = lon
        latitudes.append(lat)
        longitudes.append(lon)

## Region Mapping

In [58]:
LATITUDE_MILES = 69
LONGITUDE_MILES = 54.6

vertical_distance = (max(latitudes) - min(latitudes))*LATITUDE_MILES
horizontal_distance = (max(longitudes) - min(longitudes))*LONGITUDE_MILES

In [59]:
# Assuming we want to make 400 squares, how should we partition the vertically/horizontally
NUM_REGIONS = 200
square_len = (vertical_distance*horizontal_distance/NUM_REGIONS)**0.5

In [60]:
square_len

3.016163898519939

In [61]:
latitude_delta = square_len/LATITUDE_MILES
LATITUDE_DELTA = square_len/LATITUDE_MILES
longitude_delta = square_len/LONGITUDE_MILES
LONGITUDE_DELTA = square_len/LONGITUDE_MILES
longitude_delta

0.05524109704248972

In [62]:
lat_lon = zip(latitudes, longitudes)

In [63]:
min_lat = min(latitudes)
min_lon = min(longitudes)
max_lat = max(latitudes)
max_lon = max(longitudes)
nrows = math.ceil((max_lat - min_lat) / latitude_delta)
ncols = math.ceil((max_lon - min_lon) / longitude_delta)
def map_point_to_region(latitude, longitude):
    return math.floor((latitude-min_lat)/latitude_delta) * ncols  + math.floor((longitude-min_lon)/longitude_delta)

In [64]:
def get_representative(region_num): # 8 cols per row, # R10 --> row_num = 1, col_num = 2
    row_num = region_num//ncols # Correct
    col_num = region_num - row_num*ncols
    lat = min_lat + row_num * latitude_delta + 0.5*latitude_delta
    lon = min_lon + col_num * longitude_delta + 0.5*longitude_delta
    return [lon, lat]

In [65]:
map_point_to_region(30.084446, -97.702885)

23

In [67]:
get_representative(13)

[-97.42491028992639, 30.047122460134204]

In [68]:
valid_regions = set()
for lat, lon in zip(latitudes, longitudes):
    valid_regions.add(map_point_to_region(lat, lon))

In [69]:
valid_regions = list(valid_regions)
valid_regions.sort()

In [70]:
region_coords = [get_representative(i) for i in range(NUM_REGIONS)]

## Batch calls of distance matrix

In [71]:
# Prepare data for api call

locations = region_coords
# locations = region_coords + ems_coords + hospital_coords
durations = [[0 for i in range(200)] for j in range(NUM_REGIONS)]

In [72]:
batch_size = 1
num_batches = math.ceil(len(valid_regions) / batch_size)
num_batches_per_round = math.ceil(num_batches/4)
current_branch = 3
remainder = len(valid_regions) % batch_size
print(num_batches)

134


In [73]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624884518d9d4075408e95e3401165976247',
#     'Authorization': '5b3ce3597851110001cf62485eef3a0ab19d4b3e8ad2d8c88dca3d19',
    'Content-Type': 'application/json; charset=utf-8'
}

In [74]:
# for i in range(num_batches_per_round*current_branch, min(num_batches_per_round*current_branch+num_batches_per_round, len(valid_regions))):
#     print("batch ", i)
#     start = i * batch_size
#     sources = list(range(start, start + batch_size))

#     # handle last partial batch
#     if(sources[batch_size - 1] >= len(locations)):
#         sources = sources[:remainder]

#     locations = [get_representative(valid_regions[j]) for j in range(len(valid_regions))]

#     body = {"locations": locations, "sources": sources}
#     call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

#     duration_batch = call.json()['durations']

#     for j in range(len(duration_batch)):
#         for k in range(len(duration_batch[0])):
#             durations[valid_regions[i*batch_size + j]][valid_regions[k]] = duration_batch[j][k]
#     time.sleep(0.1)


In [75]:
locations = [get_representative(i) for i in range(NUM_REGIONS)]

durations = []

for i in range(25):
    start = i * 8
    sources = list(range(start, start + 8))
    
    body = {"locations": locations, "sources": sources}
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
    
    duration_batch = call.json()['durations']
    for j in range(len(duration_batch)):
        durations.append(duration_batch[j])
        
    time.sleep(0.1)
    
pprint.pprint(durations)

[[0.0,
  1917.78,
  2057.79,
  2379.26,
  3991.0,
  3471.98,
  3814.12,
  4184.63,
  4468.9,
  4510.44,
  5966.19,
  5235.99,
  5427.24,
  6351.45,
  6579.91,
  2535.71,
  2148.78,
  2705.97,
  3414.89,
  3583.7,
  3744.2,
  3786.81,
  4407.25,
  4362.16,
  4811.0,
  5000.15,
  5028.13,
  5151.51,
  5661.09,
  5929.15,
  2331.03,
  1976.2,
  2691.27,
  3486.61,
  5030.04,
  3886.35,
  4154.82,
  4416.85,
  4562.59,
  4844.96,
  5201.5,
  5371.67,
  5298.25,
  5602.76,
  5986.81,
  3159.6,
  2450.04,
  3265.35,
  3333.37,
  3814.65,
  4195.22,
  4281.9,
  4491.57,
  4844.5,
  4672.04,
  5043.23,
  4896.13,
  5905.5,
  6018.79,
  6622.02,
  2910.67,
  2674.75,
  3150.62,
  3224.91,
  3507.32,
  3584.18,
  4114.03,
  4204.08,
  4486.16,
  4754.54,
  5678.82,
  5850.48,
  5933.11,
  6585.4,
  6937.2,
  3350.79,
  3050.17,
  3425.82,
  4273.43,
  3638.67,
  4080.24,
  4162.1,
  4224.12,
  4648.37,
  5282.88,
  5079.71,
  5441.1,
  6118.78,
  6553.99,
  7400.59,
  4949.22,
  3382.71,
  3979.

  3205.89,
  3396.83,
  6631.1,
  5159.56,
  4746.64,
  4632.89,
  4274.87],
 [4503.01,
  3761.59,
  2445.27,
  2123.75,
  2257.85,
  1530.89,
  1061.53,
  610.27,
  0.0,
  402.35,
  1858.11,
  1127.9,
  1319.15,
  2243.36,
  2471.83,
  4990.29,
  3860.99,
  4188.23,
  3288.89,
  1850.54,
  1693.17,
  1011.92,
  914.78,
  453.07,
  882.61,
  892.06,
  920.04,
  1043.42,
  1553.0,
  1821.07,
  3687.4,
  3302.67,
  2549.4,
  2959.08,
  2788.24,
  1644.55,
  1206.91,
  1029.23,
  795.77,
  911.12,
  1267.67,
  1422.86,
  1528.31,
  1527.46,
  1878.72,
  3788.76,
  3289.94,
  3450.98,
  3044.39,
  2272.28,
  1961.31,
  1452.72,
  1339.59,
  935.41,
  800.94,
  1285.32,
  1138.22,
  2147.59,
  1943.49,
  2513.94,
  3507.66,
  3266.81,
  3082.63,
  2723.67,
  2633.22,
  2021.67,
  1846.71,
  1395.9,
  1295.95,
  1134.85,
  1851.17,
  2092.57,
  2105.46,
  2510.1,
  2829.11,
  3933.45,
  3296.58,
  3454.45,
  3374.68,
  2415.63,
  2367.5,
  1881.37,
  1817.3,
  1481.55,
  1631.63,
  1252.06,


  4139.42,
  3970.1,
  4138.0,
  3546.9,
  3372.36,
  3409.17,
  3330.26,
  3520.82,
  3657.03,
  3847.96,
  7265.26,
  5793.72,
  5380.8,
  5267.04,
  4909.03],
 [6352.88,
  5611.46,
  4295.14,
  3973.63,
  4227.43,
  3500.48,
  2911.41,
  2661.07,
  2281.49,
  1840.99,
  2658.87,
  1846.55,
  1079.59,
  0.0,
  1043.3,
  6840.17,
  5096.58,
  6038.11,
  5138.77,
  3820.13,
  3662.75,
  2981.51,
  2921.53,
  2422.66,
  2320.68,
  1742.55,
  1511.21,
  1273.73,
  1192.17,
  837.63,
  4922.99,
  4538.26,
  4188.31,
  4362.39,
  4575.44,
  3431.75,
  3098.9,
  3035.98,
  2740.44,
  2342.84,
  2076.92,
  1739.52,
  1351.17,
  1286.48,
  1230.28,
  4933.16,
  4474.68,
  4595.38,
  4188.79,
  3675.59,
  3572.1,
  3147.15,
  2979.44,
  2727.1,
  2164.62,
  2148.9,
  1591.73,
  2056.34,
  1702.51,
  1892.03,
  4652.06,
  4411.21,
  4227.03,
  3868.07,
  3777.62,
  3166.07,
  2991.11,
  2540.3,
  2440.35,
  2284.38,
  3076.75,
  2730.97,
  2182.36,
  2269.13,
  2207.21,
  5077.85,
  4440.98,
  

  3092.3,
  3814.34,
  4166.73,
  2910.19,
  2468.92,
  2626.79,
  2547.02,
  1790.99,
  1844.52,
  1488.4,
  1550.42,
  1947.97,
  2618.47,
  2238.9,
  2600.29,
  3277.97,
  3713.18,
  4697.37,
  4508.62,
  2801.47,
  2978.32,
  2267.17,
  2066.52,
  2138.71,
  2352.71,
  1527.06,
  2057.18,
  2558.53,
  2577.52,
  2855.4,
  3381.01,
  3715.09,
  3515.38,
  3871.32,
  3337.62,
  2701.89,
  2387.38,
  3205.8,
  3176.39,
  2234.04,
  1810.06,
  2187.96,
  2414.14,
  2415.5,
  2608.85,
  2868.43,
  3315.63,
  3431.57,
  4788.35,
  3070.97,
  3196.31,
  4537.84,
  2768.95,
  2740.32,
  2317.24,
  2076.18,
  2206.14,
  2626.12,
  2733.04,
  3459.55,
  3703.93,
  3571.95,
  3690.61,
  3694.6,
  3809.76,
  3584.67,
  4183.42,
  4504.62,
  2938.35,
  2543.32,
  2307.18,
  2246.64,
  2676.86,
  2768.27,
  3238.25,
  3343.6,
  4209.5,
  4571.17,
  3923.56,
  4673.15,
  6399.06,
  3915.86,
  3723.09,
  2965.86,
  2555.73,
  2739.43,
  2478.08,
  2563.72,
  3128.76,
  3230.91,
  3739.11,
  3947.6

  2101.28,
  1926.31,
  1475.51,
  1375.56,
  1214.46,
  1930.78,
  2172.18,
  1937.13,
  2023.89,
  2342.9,
  4013.06,
  3376.19,
  3534.06,
  3454.29,
  2495.24,
  2447.11,
  1960.98,
  1896.91,
  1561.92,
  1711.23,
  1331.67,
  1693.05,
  2370.74,
  2098.19,
  2985.98,
  5625.82,
  3708.73,
  3682.58,
  2971.42,
  2664.39,
  2611.3,
  2825.29,
  2081.45,
  2118.52,
  1812.3,
  1670.29,
  1948.17,
  2473.77,
  2565.12,
  2611.02,
  4778.59,
  4041.87,
  3406.15,
  3091.64,
  3910.05,
  3648.97,
  2706.63,
  2364.45,
  2060.5,
  1875.3,
  1508.27,
  1704.49,
  1964.07,
  2411.26,
  2527.21,
  5492.6,
  3775.22,
  3900.57,
  4771.64,
  3473.21,
  3212.91,
  2789.83,
  2336.08,
  2376.36,
  2206.75,
  1825.81,
  2555.19,
  2799.56,
  2667.59,
  2786.25,
  4398.85,
  4514.01,
  4288.93,
  4417.22,
  4738.42,
  3229.7,
  2777.12,
  2540.98,
  2574.09,
  2341.55,
  1861.03,
  2447.57,
  2439.23,
  3305.13,
  3666.81,
  4627.81,
  5377.4,
  6632.86,
  4149.66,
  3956.89,
  3199.66,
  2789.

...

  3081.71,
  3477.99,
  3470.75,
  3640.91,
  3536.77,
  3841.28,
  4262.05,
  4585.79,
  4127.32,
  4248.02,
  3841.43,
  3247.51,
  3144.02,
  3115.86,
  2985.19,
  3068.36,
  2910.55,
  3281.75,
  3134.65,
  4144.02,
  4257.31,
  4557.65,
  4304.69,
  4063.84,
  3879.66,
  3520.71,
  3430.25,
  2818.7,
  2740.4,
  2646.4,
  2622.01,
  2993.06,
  3534.8,
  4089.0,
  3789.09,
  4685.53,
  4242.27,
  4724.23,
  4087.36,
  4251.49,
  4171.71,
  3212.67,
  3164.18,
  2495.09,
  2304.01,
  2497.22,
  2708.05,
  2935.69,
  3297.07,
  3531.35,
  4018.69,
  4632.49,
  6343.25,
  3871.69,
  3828.37,
  3117.21,
  2969.64,
  2916.54,
  2834.2,
  2025.94,
  2259.61,
  2329.86,
  2646.84,
  2876.74,
  3575.89,
  3647.38,
  3447.67,
  4941.55,
  4042.39,
  3175.93,
  2644.72,
  3242.34,
  2465.29,
  2249.6,
  1902.42,
  1937.98,
  2337.72,
  2340.24,
  2541.14,
  2800.72,
  3247.92,
  3363.86,
  5493.12,
  3775.74,
  3670.35,
  1419.91,
  2107.01,
  1741.81,
  2177.77,
  1675.4,
  1935.68,
  2519.

  1168.01,
  1515.93,
  1775.51,
  2222.71,
  2338.65,
  5832.09,
  4114.71,
  4169.39,
  3515.4,
  2606.05,
  2280.1,
  2116.65,
  1613.35,
  1575.53,
  1578.9,
  1064.75,
  2366.63,
  2510.56,
  2479.03,
  2366.04,
  4738.34,
  4853.5,
  4628.42,
  3160.97,
  3482.18,
  1964.16,
  1689.22,
  1605.58,
  1267.79,
  1066.98,
  1076.46,
  1314.28,
  1618.04,
  2702.92,
  2464.84,
  4967.3,
  5716.89,
  5376.62,
  2893.41,
  2700.64,
  1928.09,
  1533.29,
  1406.77,
  1067.75,
  922.65,
  814.39,
  1228.4,
  1180.51,
  1641.62,
  2135.16,
  5296.31,
  3785.97,
  3832.78,
  3508.06,
  2449.51,
  2095.19,
  1624.03,
  1724.39,
  1156.32,
  703.29,
  0.0,
  811.35,
  1349.41,
  1392.54,
  1862.2,
  4318.96,
  4571.26,
  4228.65,
  3464.64,
  2763.73,
  2192.76,
  2023.44,
  1906.89,
  1315.79,
  954.33,
  934.35,
  855.45,
  1046.0,
  1182.21,
  1373.15,
  5211.22,
  3739.69,
  3326.76,
  3213.01,
  2962.37],
 [6071.42,
  5383.03,
  4329.44,
  4007.93,
  3868.64,
  3382.74,
  3305.36,
  2854

  5188.62,
  6847.32,
  5081.32,
  6194.88,
  5043.94,
  3850.9,
  3725.21,
  3526.17,
  3908.63,
  3565.45,
  3847.81,
  3971.6,
  4405.35,
  4234.13,
  4382.03,
  4537.86,
  4907.73,
  4523.0,
  4092.34,
  4266.42,
  4479.47,
  3335.78,
  3399.71,
  3481.85,
  3413.81,
  3810.1,
  3802.85,
  3973.02,
  3868.87,
  4173.38,
  4594.15,
  4917.9,
  4459.42,
  4580.12,
  4173.53,
  3579.62,
  3476.13,
  3447.96,
  3317.29,
  3400.47,
  3242.66,
  3613.85,
  3466.75,
  4476.13,
  4589.41,
  4889.76,
  4636.8,
  4395.95,
  4211.77,
  3852.81,
  3762.36,
  3150.81,
  3072.5,
  2978.5,
  2954.11,
  3325.17,
  3885.39,
  4421.1,
  4139.68,
  5036.12,
  4574.37,
  5062.59,
  4425.72,
  4583.59,
  4503.82,
  3544.77,
  3496.28,
  2827.19,
  2636.11,
  2829.33,
  3040.15,
  3286.28,
  3647.67,
  3863.45,
  4350.79,
  4964.59,
  6675.35,
  4394.98,
  4351.65,
  3640.5,
  3301.75,
  3248.65,
  3166.3,
  2358.05,
  2591.71,
  2661.97,
  2999.38,
  3208.85,
  3908.0,
  3979.48,
  3779.78,
  5464.83,


  3163.23,
  2933.24,
  2494.12,
  2712.18,
  4014.87,
  3284.67,
  3385.13,
  3824.5,
  3974.57,
  6279.24,
  4513.24,
  5626.8,
  4475.85,
  3282.82,
  3157.12,
  2716.31,
  2882.92,
  2506.26,
  2563.32,
  2687.1,
  3120.86,
  3020.07,
  3167.97,
  3323.81,
  4339.64,
  3954.92,
  3524.25,
  3698.33,
  3911.39,
  2767.7,
  2589.84,
  2671.98,
  2597.71,
  2525.61,
  2539.74,
  2709.91,
  2654.82,
  2959.33,
  3380.1,
  4349.81,
  3891.34,
  4012.04,
  3605.45,
  3011.53,
  2908.04,
  2638.09,
  2507.43,
  2584.37,
  2021.89,
  2399.79,
  2252.69,
  3262.07,
  3299.43,
  3632.33,
  4068.71,
  3827.86,
  3643.68,
  3284.73,
  3194.27,
  2582.72,
  2504.42,
  2168.63,
  2325.19,
  2117.98,
  2462.46,
  3207.05,
  2716.75,
  3613.19,
  3316.95,
  4494.51,
  3857.64,
  4015.51,
  3935.73,
  2976.69,
  2928.2,
  2259.11,
  2063.36,
  2019.46,
  2188.46,
  1863.35,
  2224.74,
  2606.03,
  3093.37,
  3707.17,
  6107.27,
  4036.1,
  3992.78,
  3281.63,
  2879.24,
  2826.14,
  2743.8,
  1790.

  3517.46],
 [6725.07,
  6036.67,
  4983.08,
  4661.57,
  4522.28,
  4036.38,
  3959.0,
  3507.76,
  3068.63,
  3286.69,
  4589.38,
  3859.18,
  3959.64,
  4399.01,
  4439.15,
  7111.39,
  5345.39,
  6458.96,
  5308.01,
  4114.98,
  3989.28,
  3517.41,
  3457.43,
  3080.77,
  3137.83,
  3261.62,
  3695.37,
  3594.58,
  3742.48,
  3824.48,
  5171.8,
  4787.08,
  4356.41,
  4530.49,
  4743.54,
  3599.86,
  3501.13,
  3571.88,
  3172.22,
  3100.12,
  3114.25,
  3284.42,
  3229.33,
  3533.84,
  3505.09,
  5181.97,
  4723.5,
  4844.2,
  4437.6,
  3843.69,
  3740.2,
  3549.39,
  3418.72,
  3158.88,
  2596.4,
  2974.31,
  2827.21,
  3836.58,
  3134.57,
  3127.27,
  4900.87,
  4660.02,
  4475.84,
  4116.88,
  4026.43,
  3414.88,
  3336.58,
  3047.65,
  2899.7,
  2692.5,
  3036.98,
  3781.56,
  3011.71,
  3329.49,
  2811.88,
  5326.66,
  4689.79,
  4847.67,
  4767.89,
  3808.85,
  3760.36,
  3091.26,
  2900.4,
  2852.25,
  2762.98,
  2437.86,
  2799.25,
  3080.1,
  2516.01,
  3129.8,
  6939.42,

In [76]:
for i in range(len(durations)):
    for j in range(len(durations[0])):
        if i != j:
            continue
        durations[i][j] = 300


In [77]:
for i in range(len(durations)):
    for j in range(len(durations[0])):
        if durations[i][j] is None:
            durations[i][j] = 300

In [31]:
durations

[[300,
  2780.06,
  3076.14,
  2866.23,
  300,
  3585.91,
  3954.72,
  4019.49,
  4424.08,
  4533.83,
  4827.96,
  5704.42,
  5496.52,
  5389.19,
  5980.38,
  5867.19,
  1474.4,
  1761.72,
  2448.15,
  2967.23,
  3466.57,
  4892.0,
  4446.27,
  4217.83,
  4540.15,
  4793.06,
  5539.37,
  5293.64,
  6971.65,
  5325.31,
  5755.22,
  6740.29,
  2756.68,
  2241.51,
  2629.63,
  3055.35,
  3287.96,
  3572.74,
  3886.43,
  4306.79,
  4456.94,
  4594.22,
  4986.98,
  4801.76,
  5073.5,
  6659.79,
  6270.23,
  6981.82,
  3353.65,
  2887.52,
  2700.19,
  3588.75,
  3140.53,
  3433.56,
  3669.55,
  4005.14,
  4211.28,
  4780.2,
  4709.61,
  5348.13,
  6514.8,
  6130.59,
  6492.45,
  7021.35,
  3380.52,
  3255.27,
  3371.6,
  3409.73,
  4367.39,
  3577.5,
  4239.59,
  4262.18,
  4423.19,
  4879.48,
  5139.88,
  5396.66,
  6054.24,
  6643.4,
  6666.06,
  6844.23,
  5303.73,
  4339.54,
  3845.67,
  4353.55,
  4014.14,
  4040.75,
  4277.42,
  4855.3,
  4385.55,
  4908.5,
  5152.33,
  5306.25,
  6054

In [78]:
def in_grid(grid_lat, grid_lon, lat, lon):
    return (grid_lat <= lat <= grid_lat + LATITUDE_DELTA and grid_lon <= lon <= grid_lon + LONGITUDE_DELTA)

In [79]:
census_tract_grids = collections.defaultdict(set)
visited = set()
for tract_name in census_tracts_points:
    for lat, lon in census_tracts_points[tract_name]:
        region = map_point_to_region(lat, lon)
#         if region not in visited:
        census_tract_grids[tract_name].add(region)
#         visited.add(region)
    census_tract_grids[tract_name] = list(census_tract_grids[tract_name])

In [80]:
import json

d = {"latitude_min": min_lat, "longitude_min": min_lon, "latitude_max": max_lat, "longitude_max": max_lon, "latitude_step": latitude_delta, "longitude_step": longitude_delta}
d["time_matrix"] = durations
d["census_tract_region_mapping"] = dict(census_tract_grids)
d["NUM_REGIONS"] = NUM_REGIONS
# with open("grid_info_multiple.json", "w") as j:
with open("grid_info_smaller.json", "w") as j:
    json.dump(d, j)

In [158]:
l = list(census_tract_grids.values())

In [164]:
l2 = list(np.concatenate(l).ravel())

In [167]:
2046 in l2

True

In [178]:
census_tract_grids["23_07"]

[887, 888, 826, 827, 765, 766]

In [6]:
import json
with open("../austin-ems-data/grid_info_multiple.json", "r") as j:
    info = json.load(j)

In [7]:
info["time_matrix"]

dict_keys(['latitude_min', 'longitude_min', 'latitude_max', 'longitude_max', 'latitude_step', 'longitude_step', 'time_matrix', 'census_tract_region_mapping'])

In [9]:
p = pd.read_csv("austin-hospitals.csv")

In [10]:
p

,OBJECTID,OBJECTID_1,EMS_STATIONS_ID,EMS_FACILITY_LABEL,FACILITY_NAME,FACILITY_TYPE,UNIT_NAME,BASE_STATION,ADDRESS,CITY,...,STATION_CATEGORY,SHARED_FACILITY,LONGITUDE,LATITUDE,X_COORD,Y_COORD,CREATED_BY,CREATED_DATE,MODIFIED_BY,MODIFIED_DATE
0,48,48,38,DM1,Demand 01,Demand Medic Station,Demand 01,Demand 01,2006 E 4th St,Austin,...,EMS-COA,NaN,-97.721725,30.259487,3121038.219,10067789.66,David Andersen,4/25/12 0:00,NaN,NaN
1,1,1,39,DM2,Demand 02,Demand Medic Station,Demand 02,Demand 02,12010 Brodie Ln,Austin,...,TCFD-EMS,FS0046,-97.860269,30.156364,3078142.397,10029294.09,David Andersen,4/25/12 0:00,NaN,NaN
2,2,2,40,DM3,Demand 03,Demand Medic Station,Demand 03,Demand 03,1705 S Congress Ave,Austin,...,AFD-EMS,FS0006,-97.750395,30.246625,3112097.570,10062901.75,Pamela Quarles,1/12/15 0:00,PFQ,4/13/16 0:00
3,3,3,41,DM4,Demand 04,Demand Medic Station,Demand 04,Demand 04,3400 Andtree Blvd,Austin,...,TCFD-EMS,FS0404,-97.661592,30.323391,3139459.749,10091475.00,Pamela Quarles,4/29/16 0:00,PFQ,11/20/17 0:00
4,4,4,49,DM5,Demand 05,Demand Medic Station,Demand 05,Demand 05,415 W 2nd St,Austin,...,AFD-EMS,NaN,-97.748300,30.265327,NaN,NaN,Pamela Quarles,10/19/16 0:00,PFQ,8/18/20 0:00
5,5,5,50,DM6,Demand 06,Demand Medic Station,Demand 06,Station 05,5710 N Lamar Blvd,Austin,...,EMS-COA,NaN,-97.726000,30.326725,3118984.917,10092201.94,Pamela Quarles,10/19/16 0:00,PFQ,8/18/20 0:00
6,50,51,53,DM7,Demand 07,Demand Medic Station,Demand 07,Demand 07,3800 Hudson Bend Rd,Austin,...,EMS-TC,NaN,-97.929578,30.398486,3054339.747,10116850.81,Pamela Quarles,3/26/19 0:00,NaN,NaN
7,15,15,1,M01,Station 01,Medic Station,Medic 01,Station 01,3616 S 1st St,Austin,...,EMS-COA,,-97.767813,30.229774,3106740.406,10056647.33,David Andersen,4/25/12 0:00,PFQ,7/18/17 0:00
8,16,16,2,M02,Station 02,Medic Station,Medic 02,Station 02,6601 Menchaca Rd,Austin,...,AFD-EMS,FS0020,-97.805440,30.206495,3095052.826,10047911.29,David Andersen,4/25/12 0:00,PFQ,5/4/19 0:00
9,17,17,3,M03,Station 03,Medic Station,Medic 03,Station 03,1305 Red River St,Austin,...,EMS-COA,,-97.734315,30.274637,3116935.629,10073204.83,David Andersen,4/25/12 0:00,NaN,NaN


In [41]:
total = 0
for i in range(len(durations)):
    for j in range(len(durations[0])):
        if durations[i][j] != 300 and durations[i][j] < 600:
            total += 1

In [42]:
total

210

In [44]:
get_representative(145)

[-98.13035987757702, 30.49168644222989]

In [45]:
for lat, lon in census_tracts_points["24_34"]:
    region = map_point_to_region(lat, lon)
    if region == -24:
        print(lat, lon)

30.029936 -97.7166509


In [66]:
map_point_to_region(30.029936, -97.7166509)

8

In [49]:
latitude=30.029936
math.floor((latitude-min_lat)/latitude_delta) * ncols

-32

In [50]:
min_lat

30.075287